<a target="_blank" href="https://colab.research.google.com/github/WSU-CS1410-AA/cs1410-notebooks/blob/main/Notebook12-templates_and_exceptions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Exceptions

A lot of what we do as programmers involves recognizing errors, anticipating them, and handling them when they occur. In fact, errors happen all the time, and we can identify three kinds of errors:
* **Compile-time errors** which are discovered and reported by the compiler during compilation. These errors must be fixed before the program can run. Unbalanced braces or missing semicolons at the ends of statements are common examples of compile-time errors.
* **Runtime errors** which pass the compiler's test and do not show up until the program is run. Unhandled runtime errors can crash the program. For example, entering the string `"fifty five"` instead of `55` for an integer variable causes an error and, unless handled, crashes the program. These errors are harder to detect than compile-time errors. We used exceptions to throw, catch, and handle these errors
* **Logical errors** which are the worst kind of errors. These are the errors that the compiler cannot detect nor do they crash the program. Instead, your program keeps running except that it produces incorrect results. For example, mistaking May 4 (05/04/2019) for April 5 (04/05/2019) is a logical error. The only way to detect and prevent these errors is to be code carefully, be organized, and test thoroughly.

One of the early goals of C++ is to improve and standardize how errors are reported and handled over what C provides. In doing so, C++ distinguishes between checking for errors and handling them. Checking for errors means detecting when errors occur. Handling them means responding to them. C++ uses exceptions to achieve this separation. When an error is checked and detected inside a function, an exception is thrown. It's up to the program that called this function to catch the thrown exception and handle it. **In C++, an unhandled exception will crash the program.**.

Here is a simple function `divide` that takes two arguments and returns the results of dividing the first argument by the second. This is a clean and simple function. The only issue is that if `by` is 0, this function will attempt to divide by zero, which is a major error in any program.

Let's see what C++ does when that happens by running this program.

In [49]:
%%writefile ex01.cpp

#include <iostream>
#include <iomanip>
#include <fstream>
#include <string>
#include <vector>

using namespace std;

double divide(double number, double by){
    return number / by;
}

int main() {
  double m, n;

  cout << "Enter a double number: ";
  cin >> m;
  cout << "Enter another double number to divide " << m << " by: ";
  cin >> n;
  cout << m << " / "  << n << ": " << divide(m, n) << endl;
  cout << "DONE!\n";

  double y = divide(m, n);
  cout << y + 1 << endl;

  return 0;
}

Overwriting ex01.cpp


In [50]:
!g++ -std=c++17 ex01.cpp -o ex01
!./ex01

Enter a double number: 23
Enter another double number to divide 23 by: 33
23 / 33: 0.69697
DONE!
1.69697


C++  returns an invalid `inf` (for infinity) value when the second entered double number is 0. You can also see that using the returned `inf` value in calculations gives us another invalid `inf` value.

To make this function safe, we need to test for such an error and prevent it from happening. But what would we do when a division by zero is detected? What should we return?

```C++
double divide(double number, double by){
    if(by == 0){
        return ???; // Not sure what to put here
    }
        
    return number / by;
}
```
Whatever we return here has to be a special value the return of which should indicate the happening of the error. Unfortunately, we don't have a spare value to use in such situations.

Prior to C++ exceptions, we would pick an arbitrary outrageously large value and return it, which is not a good practice.

Using C++ exceptions,  all we need to do is throw **an exception** instead of return a value. But what is an exception and how do we throw it? Well, in C++ an exception could be any value or object. In this example, we could throw the value of `by` back as the exception. To throw an exception, C++ provides the `throw` keyword.

Here is a revised version of the above program using `throw`.

In [51]:
%%writefile ex02.cpp

#include <iostream>
#include <iomanip>
#include <fstream>
#include <string>
#include <vector>

using namespace std;

double divide(double number, double by){
  if(by == 0){
    throw by;
  }

  return number / by;
}

int main() {
  double m, n;

  cout << "Enter a double number: ";
  cin >> m;
  cout << "Enter another double number to divide " << m << " by: ";
  cin >> n;
  cout << m << " / "  << n << ": " << divide(m, n) << endl;
  cout << "DONE!\n";

  double y = divide(m, n);
  cout << y + 1 << endl;

  return 0;
}

Overwriting ex02.cpp


Notice how we used a simple `if` conditional to test for the error and throw an exception. Let's run this new program and see what happens when an exception is thrown.

In [52]:
!g++ -std=c++17 ex02.cpp -o ex02
!./ex02

Enter a double number: 44
Enter another double number to divide 44 by: 33
44 / 33: 1.33333
DONE!
2.33333


As you can see, the previous program crashed when the second entered double number is 0 and the exception is thrown. This program crash is because the thrown value/error was not caught.

How do we prevent this program from crashing when an exception is thrown? The answer is to catch the thrown exception and handle it gracefully.

To catch and handle exceptions, C++ provides the `try-catch` blocks. Here is the above program with `try` and `catch`.

Here is a revised program that does just that.

In [53]:
%%writefile ex03.cpp

#include <iostream>
#include <iomanip>
#include <fstream>
#include <string>
#include <vector>

using namespace std;

double divide(double number, double by){
  if(by == 0){
    throw by;
  }

  return number / by;
}

int main() {
  double m, n;

  cout << "Enter a double number: ";
  cin >> m;
  cout << "Enter another double number to divide " << m << " by: ";
  cin >> n;
  try{
    cout << m << " / "  << n << ": " << divide(m, n) << endl;
    cout << "DONE!\n";
  }catch(double ex){
      cerr << "Exception occurred: cannot divide by zero.";
  }

  return 0;
}

Overwriting ex03.cpp


In [54]:
!g++ -std=c++17 ex03.cpp -o ex03
!./ex03

Enter a double number: 12
Enter another double number to divide 12 by: 32
12 / 32: 0.375
DONE!


Inside the `try{...}` block, we place the code that can throw an exception (the call to the `divide()` function in this example), and inside the `catch(double ex){...}` block, we handle the thrown exception (by reporting the error in this example). Notice that data type `double` in the `catch(double ex){...}` block matches that of the value/object thrown inside `divide1`. Notice also that when the exception is thrown, the `"DONE!"` string is not printed out. This is because inside the `try` block, when an exception is thrown, control is transferred to the corresponding `catch` block and the rest of the `try` block is skipped over.

## CODING CHALLENGE 1
Refactor the constructor of the following class so that if the width argument is negative, an exception is thrown.

Then in the `main` function, use `try` and `catch` to create two square objects one with a positive width and another with a negative width. On each object call the show function. When an exception is thrown, catch it and print an appropriate error message to the standard error (using `cerr`).

In [55]:
%%writefile ch01.cpp
#include <iostream>
#include <stdexcept>  // for std::invalid_argument

using namespace std;

class Square {
private:
    int x, y, width;
public:
    Square(int x, int y, int w) : x(x), y(y) {
        if (w < 0) {
            throw invalid_argument("Width cannot be negative");
        }
        width = w;
    }

    void show() {
        cout << width << "x" << width << " SQUARE @(" << x << "," << y << ")" << endl;
    }
};

int main() {
    try {
        Square s1(10, 20, 5);
        s1.show();
    } catch (const invalid_argument& e) {
        cerr << "Error creating square s1: " << e.what() << endl;
    }

    try {
        Square s2(30, 40, -7);  // This should throw
        s2.show();
    } catch (const invalid_argument& e) {
        cerr << "Error creating square s2: " << e.what() << endl;
    }

    return 0;
}


Overwriting ch01.cpp


In [56]:
!g++ -std=c++17 ch01.cpp -o ch01
!./ch01

5x5 SQUARE @(10,20)
Error creating square s2: Width cannot be negative


## Using the `runtime_error` class

Instead of throwing arbitrary values for exceptions, we can use the standard  `runtime_error` exception class the C++ provides via the header the file `<stdexcept>`. Here is another version of the `divide` function.

In [57]:
%%writefile ex04.cpp

#include <iostream>
#include <stdexcept>
#include <iomanip>
#include <fstream>
#include <string>
#include <vector>

using namespace std;

double divide(double number, double by){
  if(by == 0){
    throw runtime_error("divide by zero");
  }

  return number / by;
}

int main() {
  double m, n;

  cout << "Enter a double number: ";
  cin >> m;
  cout << "Enter an other double number to divide " << m << " by: ";
  cin >> n;
  try {
    cout << m << " / "  << n << ": " << divide(m, n) << endl;
    cout << "DONE!\n";
  }catch(runtime_error& ex){
      cerr << "Exception occurred: " << ex.what();
  }

  return 0;
}

Overwriting ex04.cpp


Using the `runtime_error` allows us to provide a message describing what the thrown exception is about. Here is how we handle this exception. Notice the use of the `what()` function of the thrown `runtime_error` exception to get the exception message.

Here is the output of the program:

In [58]:
!g++ -std=c++17 ex04.cpp -o ex04
!./ex04

Enter a double number: 87
Enter an other double number to divide 87 by: 31
87 / 31: 2.80645
DONE!


## Using own exception class

Alternatively, we can create our own exception class. Here is a simple exception class that extends or inherits from the standard `runtime_error` class.

In [59]:
%%writefile ex05.cpp

#include <iostream>
#include <stdexcept>
#include <iomanip>
#include <fstream>
#include <string>
#include <vector>

using namespace std;

class DivisionByZero : public runtime_error{
public:
    DivisionByZero(): runtime_error("divide by zero"){}
};

Overwriting ex05.cpp


Here is a version of the `divide` function using this exception:

In [60]:
%%writefile -a ex05.cpp

double divide(double number, double by){
    if(by == 0){
        throw DivisionByZero();
    }

    return number / by;
}

Appending to ex05.cpp


And here is the code to try and catch it:

In [61]:
%%writefile -a ex05.cpp

int main() {
  double m, n;

  cout << "Enter a double number: ";
  cin >> m;
  cout << "Enter an other double number to divide " << m << " by: ";
  cin >> n;
  try {
      cout << m << " / "  << n << ": " << divide(m, n) << endl;
      cout << "DONE!\n";
  }catch(DivisionByZero& ex){
      cerr << "Exception occurred: " << ex.what();
  }

  return 0;
}

Appending to ex05.cpp


In [62]:
!g++ -std=c++17 ex05.cpp -o ex05
!./ex05

Enter a double number: 25
Enter an other double number to divide 25 by: 66
25 / 66: 0.378788
DONE!


## Catching more than one exception

In C++, we can have more than one `catch` block for the same `try` block. This is because the code inside the `try` block can throw more than one exception or no exceptions at all. Here is an example with three `catch` blocks.

```c++
try {
   // Code to execute
} catch (invalid_argument& e){
   // Handle invalid argument
} catch (runtime_error& e){
   // Handle exception
} catch (...){
   // Handle anything else
}
```

The third `catch` block has `...` between its parentheses to catch everything else that is not `invalid_argument` or `runtime_error`.

We can also use the `catch(...)` block by itself if we want to catch all possible exceptions without distinguishing one from another.

```c++
try {
   // Code to execute
} catch (...){
   // Code to handle all exceptions
}
```

# Templates

## Function templates

In programming, we often come across functions or classes that are generic in nature; meaning that they can be applied the same way to objects of different data types. Take, for instance, the following `lesserOf` function, which takes two double arguments and returns the lesser of the two.

```cpp
double lesserOf(double n, double m){
    return n < m ? n : m;
}
```

If we were to implement this `lesserOf` function for integers or strings, we would end up with the same function duplicated three times; only the data types of the arguments would change.

How can we make functions like `lesserOf` generic and without repeating the same logic over and over? C++'s answer to this important question is **generic functions** or **function templates**. According to cplusplus.com,  "**Function templates** are special functions that can operate with generic types. This allows us to create a function template whose functionality can be adapted to more than one type or class without repeating the entire code for each type".

To define a **function template** we start with the keyword `template` followed by the angle brackets `<>` and in between them the keyword `typename` and a type identifier of your choosing such as `T` or `TypeID`. We do all this in front of the function declaration or definition either in the same line or in the line above.

You can also replace the keyword `typename` with the keyword `class`. I personally recommend using `typename`. That way the keyword `class` is only used to create classes.

Here is the lesserOf function as a function template:

```cpp
template<typename T>
T lesserOf(T n, T m){
    return n < m ? n : m;
}
```

or like this (with the keyword `typename` replaced by the keyword `class`):

```c++
template<class T>
T lesserOf(T n, T m){
    return n < m ? n : m;
}
```

Here we used the generic type `T` for the return type and arguments. This type `T` is not a real type, the compiler will replace it with the actual data type when the function is called.

When the compiler sees a **function template** like this, it takes note of it and when it sees a call to it, it generates a specific version of the function template based on the data types of its arguments.  In other words, the call:

```cpp
cout << lesserOf<int>(12, 9) << endl;
```

will cause the compiler to generate a function like  this:

```c++
int lesserOf(int n, int m){
    return n < m ? n : m;
}
```

because the arguments of the call were integers. Notice the use of the angle brackets `<>` after the function name to pass the actual data type of the arguments.

Similarly, the call:

```cpp
cout << lesserOf<double>(1.2, 9.8) << endl;
```

will cause the compiler to generate this function because of the double arguments:

```c++
double lesserOf(double n, double m){
    return n < m ? n : m;
}
```

And to make it easier for the programmers, the compiler does not require the angle brackets `<>` as long as it can detect the types from the function arguments. That means, the previous calls to `lesserOf` can be rewritten without the angle brackets `<>` and the compiler will be able to deduce the data type from the passed arguments. For example, the statements:

```cpp
cout << lesserOf(12, 9) << endl;
cout << lesserOf(1.2, 9.8) << endl;
```

will compile without errors and output:

```txt
9
1.2
```

## CODING CHALLENGE 2

Here is a function that compares two integers `a` and `b` and returns 0 if `a` equals `b`, 1 if `a` is greater than `b`, and -1 if `a` is less than `b`.

```c++
int compareTo(double a, double b){
    if(a > b) return 1;
    else if (a == b) return 0;
    else return -1;
}
```

Copy this function to the program below and make it a function template that works for different arguments than double. Notice that this function should always return `int`. In the `main` function, test your function template using integer, double, and string arguments with and without the angle brackets `<>` after the function name.

In [63]:
%%writefile ch02.cpp
#include <iostream>
#include <string>

using namespace std;

// Function template
template <typename T>
int compareTo(T a, T b) {
    if (a > b) return 1;
    else if (a == b) return 0;
    else return -1;
}

int main() {
    // Integer comparisons
    cout << "compareTo(5, 3) = " << compareTo(5, 3) << endl;
    cout << "compareTo(5, 5) = " << compareTo(5, 5) << endl;
    cout << "compareTo(3, 5) = " << compareTo(3, 5) << endl;

    // Double comparisons
    cout << "compareTo(3.14, 2.71) = " << compareTo(3.14, 2.71) << endl;
    cout << "compareTo(3.14, 3.14) = " << compareTo(3.14, 3.14) << endl;
    cout << "compareTo(2.71, 3.14) = " << compareTo(2.71, 3.14) << endl;

    // String comparisons
    cout << "compareTo(\"apple\", \"banana\") = " << compareTo(string("apple"), string("banana")) << endl;
    cout << "compareTo(\"apple\", \"apple\") = " << compareTo(string("apple"), string("apple")) << endl;
    cout << "compareTo(\"banana\", \"apple\") = " << compareTo(string("banana"), string("apple")) << endl;

    // Using explicit template arguments
    cout << "compareTo<int>(7, 4) = " << compareTo<int>(7, 4) << endl;
    cout << "compareTo<double>(5.5, 5.5) = " << compareTo<double>(5.5, 5.5) << endl;
    cout << "compareTo<string>(\"cat\", \"dog\") = " << compareTo<string>("cat", "dog") << endl;

    return 0;
}

Overwriting ch02.cpp


In [64]:
!g++ -std=c++17 ch02.cpp -o ch02
!./ch02

compareTo(5, 3) = 1
compareTo(5, 5) = 0
compareTo(3, 5) = -1
compareTo(3.14, 2.71) = 1
compareTo(3.14, 3.14) = 0
compareTo(2.71, 3.14) = -1
compareTo("apple", "banana") = -1
compareTo("apple", "apple") = 0
compareTo("banana", "apple") = 1
compareTo<int>(7, 4) = 1
compareTo<double>(5.5, 5.5) = 0
compareTo<string>("cat", "dog") = -1


## More than one template type

And we are not limited to a single template type, we can have two or more types. Here is an example with two template types `T` and `S` (you can choose different names for these types if you want to). This function tries to find a given value in an array and if found return the index of the found value.

```cpp
template <typename T, typename S>
S find(T* data, T value, S size){
    for(S j = 0; j < size; j++){
        if(data[j] == value){
            return j;
        }
    }
    
    return -1;
}
```

Here is how to call this function.

```cpp
char vowels[] = {'A', 'E', 'I', 'O', 'U', 'Y'};
cout << find(vowels, 'X', 6) << endl;             
cout << find<char, unsigned>(vowels, 'U', 6) << endl;
```

which returns:

```txt
-1
4
```

## Class templates

**Generic programming** in C++ does not stop at **function templates**, it also supports **class templates**. Here is an example using the vector class.

In [65]:
%%writefile ex06.cpp

#include <iostream>
#include <vector>

using namespace std;

int main(){
  vector<double> grades = {89.5, 91.0, 96.25, 75.5, 94.75};
  double average = 0;
  for(double grade : grades){
      average += grade / grades.size();
  }

  cout << "Grade average: " << average << endl;

  return 0;
}

Overwriting ex06.cpp


In [66]:
!g++ -std=c++17 ex06.cpp -o ex06
!./ex06

Grade average: 89.4


Did you notice the angle brackets `<>` after `vector`? That tells us that `vector` is a **class template**. Other class templates that the C++ standard library comes with include `map`, `array`, `queue` and `stack`, to name a few. As a matter of fact, class templates are critical to implementing the classic data structures and algorithms that are the bedrock of computer science.

Like function templates, we define class templates using the keyword `template` followed by angle brackets `<>` with one or more type declarations in between; each preceded by the `typename` or `class` keyword. Here is an example class template `A` with a single type argument `T`.

In [67]:
%%writefile ex07.cpp

#include <iostream>

using namespace std;

template <typename T>
class A{
public:
  A(T a): m(a){}
  void show(){
    cout << "My A-number is: " << m << endl;
  }
  ~A(){}
private:
  T m;
};

Overwriting ex07.cpp


To create an object from a class template, we must provide the type between the angle brackets `<>` as you see here. In other words, the angle brackets `<>` are not optional as was the case with function templates.

Here is a program with three different objects of this `A` class: one in the stack and two in the heap using the `new` operator. Because each of these objects has a different type, the compiler will create thee different actual classes from this class template: one for `int`, another for `double` and the last one for `string`.

In [68]:
%%writefile -a ex07.cpp

int main(){
  A<int> a(56);
  a.show();

  A<double> *b = new A<double>(44.5);
  b->show();

  A<string> *c = new A<string>("James");
  c->show();

  delete b, c;

  return 0;
}

Appending to ex07.cpp


In [69]:
!g++ -std=c++17 ex07.cpp -o ex07
!./ex07

My A-number is: 56
My A-number is: 44.5
My A-number is: James


Here is another class template `BC` with two type arguments followed by two objects of this class.

In [70]:
%%writefile ex08.cpp

#include <iostream>

using namespace std;

template <typename E, typename T>
class BC{
public:
    BC(E b, T c): b(b), c(c){}
    void print(){
        cout << "b: " << b << ", "
             << "c: " << c << endl;
    }
    ~BC(){}
private:
    E b;
    T c;
};

Overwriting ex08.cpp


In [71]:
%%writefile -a ex08.cpp

int main(){
  BC<int, double> bc(10, 89.05);
  bc.print();

  BC<string, short> *bcptr = new BC<string, short>("Washington", 10);
  bcptr->print();
  delete bcptr;

  return 0;
}

Appending to ex08.cpp


In [72]:
!g++ -std=c++17 ex08.cpp -o ex08
!./ex08

b: 10, c: 89.05
b: Washington, c: 10


## CODING CHALLENGE 3

In code cell below, write a class template named `Pair` with two type arguments `K`and `V`.  This template is for grouping two values into a pair, which could be useful in data structures like maps and dictionaries, with the first value being the key and the second being the value or meaning. This class should have two private data members: one named `key` of the first type argument and the second named `value` of the second type argument. This class should have a two-argument constructor, a destructor, and the following four member functions (getters/setters):
* `K getKey()` for returning the key the pair.
* `void setKey(K k)` for setting the key of the pair to the passed argument.
* `V getValue()` for returning the value the pair.
* `void setValue(V v)` for setting the value of the pair to the passed argument.

In the `main` function, create four pair objects each with a different type combinations. Two of these objects should be in the stack and two in the heap (using the `new` operator). On each object call at least two of the member functions above.

In [73]:
%%writefile ch03.cpp
#include <iostream>
#include <string>

using namespace std;

// Class template with two type arguments K and V
template <typename K, typename V>
class Pair {
private:
    K key;
    V value;
public:
    // Constructor
    Pair(K k, V v) : key(k), value(v) {}

    // Destructor
    ~Pair() {}

    // Getter for key
    K getKey() {
        return key;
    }

    // Setter for key
    void setKey(K k) {
        key = k;
    }

    // Getter for value
    V getValue() {
        return value;
    }

    // Setter for value
    void setValue(V v) {
        value = v;
    }
};

int main() {
    // Stack objects
    Pair<int, string> pair1(1, "One");
    Pair<string, double> pair2("Pi", 3.1415);

    // Heap objects
    Pair<char, int>* pair3 = new Pair<char, int>('A', 65);
    Pair<string, string>* pair4 = new Pair<string, string>("Country", "Nepal");

    // Using member functions
    cout << "pair1: key = " << pair1.getKey() << ", value = " << pair1.getValue() << endl;
    pair1.setValue("Uno");
    cout << "pair1 after setValue: value = " << pair1.getValue() << endl;

    cout << "pair2: key = " << pair2.getKey() << ", value = " << pair2.getValue() << endl;
    pair2.setKey("π");
    cout << "pair2 after setKey: key = " << pair2.getKey() << endl;

    cout << "pair3: key = " << pair3->getKey() << ", value = " << pair3->getValue() << endl;
    pair3->setValue(66);
    cout << "pair3 after setValue: value = " << pair3->getValue() << endl;

    cout << "pair4: key = " << pair4->getKey() << ", value = " << pair4->getValue() << endl;
    pair4->setKey("Capital");
    cout << "pair4 after setKey: key = " << pair4->getKey() << endl;

    // Clean up heap memory
    delete pair3;
    delete pair4;

    return 0;
}

Overwriting ch03.cpp


In [74]:
!g++ -std=c++17 ch03.cpp -o ch03
!./ch03

pair1: key = 1, value = One
pair1 after setValue: value = Uno
pair2: key = Pi, value = 3.1415
pair2 after setKey: key = π
pair3: key = A, value = 65
pair3 after setValue: value = 66
pair4: key = Country, value = Nepal
pair4 after setKey: key = Capital


## Templates with constant type arguments
Class templates can have constant type arguments. The template class `array` from the standard library, for example, requires two type arguments to create a static array: the type of the data in the array and size of the array.

```cpp
#include <array>
array<int, 2> vals = {11, 17};
```

Here is an example of using constant type arguments to create our `OwnArray`. It has two constructors: a no-argument constructor and a one-argument constructor.

In [75]:
%%writefile ex09.cpp

#include <iostream>
#include <string>

using namespace std;

template <typename T, unsigned short S>
class OwnArray {
private:
    T *data;
    int size;
public:
    OwnArray(): size(S), data(new T[S]){}
    OwnArray(T *arr): OwnArray(){
        for(int i = 0; i < size; i++){
            data[i] = arr[i];
        }
    }
    void print(){
        for (int i = 0; i < size; i++){
            cout << " " << *(data + i);
        }
        cout << endl;
    }
    ~OwnArray(){
        delete[] data;
    }
};

Overwriting ex09.cpp


Notice how the one-argument constructor delegates the initialization of the `data` member to the no-argument constructor. Here is an example of using this class:

In [76]:
%%writefile -a ex09.cpp

int main(){
  string arr[] = {"ma", "ma", "mia"};
  OwnArray<string, 3> oa(arr);
  oa.print();

  return 0;
}

Appending to ex09.cpp


In [77]:
!g++ -std=c++17 ex09.cpp -o ex09
!./ex09

 ma ma mia


## Templates with separted definition and implementation
And like regular classes, we can separate the definition of class template from its implementation.

It's not straightforward, however, to separate a C++ template class into .h and .cpp files in the same way as non-template classes. This is because the compiler needs to know the complete definition of the template class at the point of instantiation, which typically happens in the testing code.

As a result, we usually make the separation in the `.h` file.

Here is an example, revising the `OwnArray` class above.

First, the class definition and implemenation, separated but saved into the `.h` file.

In [78]:
%%writefile own_array.h

#ifndef __OWN_ARRAY_H__
#define __OWN_ARRAY_H__

#include <iostream>
#include <string>

// Class definition
template <typename T, int S>
class OwnArray {
private:
    T *data;
    int size;
public:
    OwnArray();
    OwnArray(T *arr);
    void print();
    ~OwnArray();
};

// Class implementation
template <typename T, int S>
OwnArray<T,S>::OwnArray(): size(S), data(new T[S]){}

template <typename T, int S>
OwnArray<T,S>::OwnArray(T *arr): OwnArray(){
    for(int i = 0; i < size; i++){
        data[i] = arr[i];
    }
}

template <typename T, int S>
void OwnArray<T,S>::print(){
    for (int i = 0; i < size; i++){
        std::cout << " " << *(data + i);
    }
    std::cout << std::endl;
}

template <typename T, int S>
OwnArray<T,S>::~OwnArray(){
    delete[] data;
}

#endif

Overwriting own_array.h


Next, we test the class in a `.cpp` file with a `main` function. Here is one such program:

In [79]:
%%writefile ex10.cpp

#include "own_array.h"

int main(){
  std::string arr[] = {"ma", "ma", "mia"};
  OwnArray<std::string, 3> ob(arr);
  ob.print();

  return 0;
}

Overwriting ex10.cpp


Let's compile and run this program.

In [80]:
!g++ -std=c++17  ex10.cpp -o ex10
!./ex10

 ma ma mia


To summarize, function and class templates are very powerful in eliminating code redundancy. They are necessary to implementing many of most commonly used classes in C++ including the vector class.


## CODING CHALLENGE 4
Refactor the following class template by separating its definition from its implementation.
```c++
template <typename T>
class A{
public:
    A(T a): m(a){}
    void show(){
        cout << "My number is: " << m << endl;
    }
    ~A(){}
private:
    T m;
};
```

In [81]:
%%writefile a.h
#ifndef __A_H__
#define __A_H__

#include <iostream>
using namespace std;

// Class template definition
template <typename T>
class A {
public:
    A(T a);       // Constructor declaration
    void show();   // show() declaration
    ~A();         // Destructor declaration
private:
    T m;
};

// Constructor implementation
template <typename T>
A<T>::A(T a) : m(a) {}

// show() implementation
template <typename T>
void A<T>::show() {
    cout << "My number is: " << m << endl;
}

// Destructor implementation
template <typename T>
A<T>::~A() {}

#endif


Overwriting a.h


In [82]:
%%writefile ch04.cpp
#include "a.h"

int main() {
    // Test with int
    A<int> a1(100);
    a1.show();

    // Test with double
    A<double> a2(3.1415);
    a2.show();

    // Test with string
    A<string> a3("Hello");
    a3.show();

    return 0;
}


Overwriting ch04.cpp


In [83]:
!g++ -std=c++17  ch04.cpp -o ch04
!./ch04

My number is: 100
My number is: 3.1415
My number is: Hello
